# Построение baseline-решений
В этом задании вам предстоит построить несколько моделей и оценить их качество. Эти модели будут служить нам в качестве baseline-решений и пригодятся сразу для нескольких задач:
1. Во-первых, на разработку baseline-модели не должно уходить много времени (это требование исходит из оценок затрат на проект в целом - большую часть времени все же нужно потратить на основное решение), процесс должен быть простым, на подавляющем большинстве этапов должны использоваться готовые протестированные инструменты. Все это приводит к тому, что baseline-модели - это дешевый способ сделать грубую оценку потенциально возможного качества модели, при построении которого вероятность допущения ошибок относительно невелика.
2. Во-вторых, использование моделей разного типа при построении baseline'ов позволяет на раннем этапе сделать предположения о том, какие подходы являются наиболее перспективными и приоритизировать дальнейшие эксперименты.
3. Наличие baseline-моделей позволяет оценить, какой прирост качества дают различные преобразования, усложнения, оптимизации и прочие активности, которые вы предпринимаете для построения финального решения.
4. Наконец, если после построение сложного решения оценка его качества будет очень сильно отличаться от оценки качества baseline-моделей, то это будет хорошим поводом поискать в решении ошибки.

Обучите 3 разные baseline-модели на полученных наборах данных и оцените их качество. На прошлой неделе вы выбрали методику оценки качества моделей на основе кросс-валидации, а также основную и вспомогательные метрики. Оцените с их помощью получившуюся модель. Обратите внимание, что под разными моделями понимаются именно разные алгоритмы классификации. Например, 2 модели, реализующие метод k ближайших соседей с разными k, будут считаться одним baseline-решением (хотя и с разными параметрами). Напоминаем, что отложенная выборка (hold-out dataset) не должна использоваться для построения и оценки baseline-моделей!

Можно (но не обязательно) рассмотреть следующий набор алгоритмов:
1. Линейная модель (например, реализация sklearn.linear_model.RidgeClassifier)
2. Случайный лес (например, реализация sklearn.ensemble.RandomForestClassifier)
3. Градиентный бустинг (например, реализация sklearn.ensemble.GradientBoostingClassifier)

В качестве решения приложите получившийся jupyther notebook. Убедитесь, что в нем присутствуют:
- все baseline-модели, которые вы построили;
- качество всех построенных моделей оценено с помощью кросс-валидации, и это понятно из текста в jupyther notebook;
- все модели оценены с помощью основной и дополнительных метрик качества.

In [16]:
import pandas as pd
import random
import numpy as np
from scipy.sparse import coo_matrix, hstack
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import RidgeClassifier
seed = 1903
first_categorial_index = 190

Загрузим train dataset, выделенный на предыдущей неделе

In [2]:
churn_data_frame = pd.read_csv("..\..\Data\churn_data_train.csv", ",")
churn_labels_frame = pd.read_csv("..\..\Data\churn_labels_train.csv")
print(churn_data_frame.shape)
print(churn_labels_frame.shape)

(27999, 230)
(27999, 1)


## Общая предобработка признаков

In [12]:
def scale_frame(frame):
    """Функция масштабирает frame на отрезок [0;1]"""
    scaler = StandardScaler()
    scaled_matrix = scaler.fit_transform(frame.as_matrix())
    return pd.DataFrame(scaled_matrix, columns=frame.columns)

def split_frame(frame, n_columns, c_columns):
    """Функция разбивает фрейм на два числовой и категориальный, а также масштабирует значения и заполняет пропуски."""
    n_frame = frame[n_columns].copy()
    c_frame = frame[c_columns].copy()
    # Посчитаем средние по колонкам
    numeric_means = n_frame.mean(axis=0, skipna=True)
    # Оставим только те колонки, в которых среднее значение не равно NaN, т.к. в таких колонках совсем нет значений
    numeric_means = numeric_means.dropna()
    dropped_numeric_colums = n_frame.columns.drop(numeric_means.index)
    n_frame = n_frame[list(numeric_means.index)]
    # Заполним пропущенные численные значения средними
    n_frame = n_frame.fillna(numeric_means, axis=0)
    # Заполним пропущенные категориальные значения строками "NaV" (Not a value)
    c_frame = c_frame.fillna("NaV")
    # Посчитаем количества уникальных значений по колонкам
    cat_unique_counts = c_frame.nunique()
    num_unique_counts = n_frame.nunique()
    # Удалим колонки с одним уникальным значением
    cat_columns_to_drop = cat_unique_counts[cat_unique_counts == 1].index
    num_columns_to_drop = num_unique_counts[num_unique_counts == 1].index
    c_frame = c_frame.drop(columns=cat_columns_to_drop)
    n_frame = n_frame.drop(columns=num_columns_to_drop)
    
    dropped_columns = np.concatenate([
        dropped_numeric_colums,
        list(num_columns_to_drop),
        list(cat_columns_to_drop)])
    return (n_frame, c_frame, dropped_columns)

Разделим коллекции на группы - числовые и категориальные.
Выполним начальные преобразования, которые мы делали на 1-й неделе
- заполним пропущенные числовые значения средними по колонке
- заполним пропущенные категориальные значения строками "NaV" (Not a Value)
- удалим колонки без значений и с только одним уникальным значением, т.к. они не сыграют роли в модели

Все эти действия производятся в функции split_frame (см. выше)

In [14]:
numeric_columns = churn_data_frame.columns[:first_categorial_index]
categorial_columns = churn_data_frame.columns[first_categorial_index:]

numeric_frame, categorial_frame, dropped_columns = split_frame(
    churn_data_frame,
    numeric_columns,
    categorial_columns)

Масштабируем числовые признаки

In [18]:
numeric_frame_scaled = scale_frame(numeric_frame)

In [25]:
def label_encode(X_categorial):
    """ Функция кодирует категории числами от 0 до n, где n количество категорий в колонке. """
    X_num = np.empty(X_categorial.shape)
    for column_number in range(X_categorial.shape[1]):
        labelEncoder = LabelEncoder()
        column = X_categorial[:,column_number]
        for idx, val in enumerate(column):
            if(not(isinstance(val, str))):
                column[idx] = "NaV"
        num_column = labelEncoder.fit_transform(column)
        for row_number, val in enumerate(num_column):
            X_num[row_number, column_number] = val
    return X_num

Закодируем категориальные признаки значениями от 0 до n. Это в любом случае понадобится, т.к. например OneHotEncoder принимает на вход целочисленную матрицу.

In [29]:
categorial_frame_labeled = pd.DataFrame(
    label_encode(categorial_frame.as_matrix()),
    columns=categorial_frame.columns)

## Построение линейной baseline модели
Для построения baseline модели не будем использовать OneHotEncoder, т.к. этот метод очень сильно увеличивает размерность пространства (см. ниже) и без отбора признаков не заработает. (По крайней мере моя машина стабильно выдает MemoryError.

Другие методы обработки категориальных признаков несколько более сложны и не укладываются в рамки baseline модели

In [31]:
one_hot = OneHotEncoder()
one_hot.fit_transform(categorial_frame_labeled.as_matrix()).shape

(27999, 51885)

In [ ]:
filled_frame = pd.concat([numeric_frame, categorial_frame], axis=1, ignore_index=True)
first_filled_categorial_index = numeric_frame.shape[1]

X_ridge = filled_frame.as_matrix()
y_ridge = churn_labels_frame.as_matrix().flatten()
categorial_columns_indices = range(first_filled_categorial_index, filled_frame.shape[1]-first_filled_categorial_index)

In [6]:
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)
train_indices, test_indices = list(skf.split(X_ridge, y_ridge))[0]
X_train = X_ridge[train_indices]
X_test = X_ridge[test_indices]
y_train = y_ridge[train_indices]
y_test = y_ridge[test_indices]
X_numeric = X_train[:,:first_filled_categorial_index]
X_categorial = X_train[:,first_filled_categorial_index:]
encoded = one_hot_encode(X_categorial)
print(X_train.shape, X_numeric.shape, X_categorial.shape, encoded.shape)

(18665, 211) (18665, 173) (18665, 38) (18665, 40569)


In [9]:
# train_matrix = np.concatenate((X_numeric, encoded.toarray()), axis=1)
# Найти способ уменьшить размерность категориальной матрицы
numeric_frame = pd.DataFrame(X_numeric)
categorial_frame = pd.DataFrame(encoded.toarray())
training_frame = pd.concat([numeric_frame, categorial_frame], axis=1, ignore_index=True)
training_frame.shape

(18665, 40742)

In [10]:
X_train_encoded = categorial_frame.as_matrix()

In [11]:
ridge = RidgeClassifier()
ridge.fit(X_train_encoded, y_train)

MemoryError: 